In [11]:
import os
import shutil

def clear_output_directory(output_dir):
    """
    Deletes all files and subdirectories in the specified output directory.
    """
    if os.path.exists(output_dir):  # Check if the directory exists
        for filename in os.listdir(output_dir):
            file_path = os.path.join(output_dir, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)  # Delete files and symbolic links
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)  # Delete directories
            except Exception as e:
                print(f"Failed to delete {file_path}: {e}")
    else:
        os.makedirs(output_dir)  # Create the directory if it does not exist

In [12]:
import albumentations as A
import cv2  # OpenCV is needed for image reading/writing

def augment_and_save_from_root(root_dir, output_dir, augment_ratio=2, save_original=True):
    """
    Perform data augmentation on a dataset organized by subject/activity type.

    Args:
        root_dir (str): Root directory containing subject/activity folders.
        output_dir (str): Directory to save augmented images.
        augment_ratio (int): Number of augmented images to generate per input image.
        save_original (bool): Whether to save the original image alongside augmented ones.
    """
    # Clear output directory
    clear_output_directory(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    # Define augmentation pipeline
    augmentation = A.Compose([
        A.RandomBrightnessContrast(p=0.3),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=7, p=0.7),
    ])

    errors = []
    augmented_data = []

    for file in os.listdir(input_dir):
        img_path = os.path.join(input_dir, file)

        # Read the image
        image = cv2.imread(img_path)
        if image is None:
            errors.append(img_path)
            continue

        # Convert image to RGB for augmentation
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Save original image if required
        filename_no_ext = os.path.splitext(file)[0]  # Get file name without extension
        if save_original:
            save_path = os.path.join(output_dir, f"{filename_no_ext}_original.jpg")
            cv2.imwrite(save_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
            augmented_data.append(save_path)

        # Generate augmented images
        for i in range(augment_ratio):
            augmented = augmentation(image=image)
            augmented_image = augmented["image"]

            # Save augmented image
            aug_save_path = os.path.join(output_dir, f"{filename_no_ext}_aug{i}.jpg")
            cv2.imwrite(aug_save_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))
            augmented_data.append(aug_save_path)

    # Log errors
    if errors:
        print("\n--- Errors During Augmentation ---")
        for err in errors:
            print(f"Could not process image: {err}")

    print(f"\nSaved {len(augmented_data)} images to {output_dir}")

In [14]:

input_dir = 'nonfall_laying_1'
output_dir = 'output'


augment_and_save_from_root(input_dir, output_dir, augment_ratio=1, save_original=True)


Saved 102 images to output
